# T81-558: Applications of Deep Neural Networks
**Class 12: Deep Learning Applications**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), School of Engineering and Applied Science, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

Tonight we will see how to apply deep learning networks to data science.  There are many applications of deep learning.  However, we will focus primarily upon data science.  For this class we will go beyond simple academic examples and see how to construct an ensemble that could potentially lead to a high score on a Kaggle competition.  We will see how to evaluate the importance of features and several ways to combine models.

Tonights topics include:

* Log Loss Error
* Evaluating Feature Importance
* The Biological Response Data Set
* Neural Network Bagging
* Nueral Network Ensemble

# Helpful Functions

These are exactly the same feature vector encoding functions from [Class 3](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class3_training.ipynb).  They must be defined for this class as well.  For more information, refer to class 3.

In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

# LogLoss Error

Log loss is an error metric that is often used in place of accuracy for classification.  Log loss allows for "partial credit" when a miss classification occurs.  For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .8 probability of A being correct, then the value -log(.8) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following table shows the logloss scores that correspond to the average predicted accuracy for the correct item. The **pred** column specifies the average probability for the correct class.  The **logloss** column specifies the log loss for that probability.


In [2]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

loss = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.075, 0.05, 0.025, 1e-8 ]

df = pd.DataFrame({'pred':loss, 'logloss': -np.log(loss)},columns=['pred','logloss'])

display(df)

,pred,logloss
0,1.000000e+00,-0.000000
1,9.000000e-01,0.105361
2,8.000000e-01,0.223144
3,7.000000e-01,0.356675
4,6.000000e-01,0.510826
5,5.000000e-01,0.693147
6,4.000000e-01,0.916291
7,3.000000e-01,1.203973
8,2.000000e-01,1.609438
9,1.000000e-01,2.302585


The table below shows the opposit.  For a given logloss, what is the average probability for the correct class.

In [3]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

loss = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 2.5, 3, 3.5, 4 ]

df = pd.DataFrame({'logloss':loss, 'pred': np.exp(np.negative(loss))},
                  columns=['logloss','pred'])

display(df)

,logloss,pred
0,0.1,0.904837
1,0.2,0.818731
2,0.3,0.740818
3,0.4,0.670320
4,0.5,0.606531
5,0.6,0.548812
6,0.7,0.496585
7,0.8,0.449329
8,0.9,0.406570
9,1.0,0.367879


# Enhanced Sklearn Compatibility

In [4]:
class ScikitLearnTFDNNClassifier:
    def __init__(self,name,hidden_units,num_classes,optimizer,steps):
        self.classifier = None
        self.model_dir = None
        self.hidden_units = hidden_units
        self.num_classes = num_classes
        self.optimizer = optimizer
        self.steps = steps
        self.name = name
        
    def __str__(self):
        return "ScikitLearnTFDNNClassifier(name='{}',hidden_units={},num_classes={},optimizer={},steps={})".format(
        self.name,self.hidden_units,self.num_classes,self.optimizer,self.steps)

    def fit(self,x,y):
        if self.classifier is None:
            self.model_dir = get_model_dir(self.name,True)

            # Create a deep neural network with 3 hidden layers of 10, 20, 5
            feature_columns = [tf.contrib.layers.real_valued_column("", dimension=x.shape[1])]
            self.classifier = learn.DNNClassifier(
                model_dir= self.model_dir,
                config=tf.contrib.learn.RunConfig(save_checkpoints_secs=30),
                hidden_units=self.hidden_units, n_classes=num_classes, feature_columns=feature_columns)

        self.classifier.fit(x,y,steps=self.steps)
        return self

    def predict(self,x):
        if self.classifier is None:
            raise ValueError('A very specific bad thing happened')

        return list(self.classifier.predict(x, as_iterable=True))

    def predict_proba(self,x):
        if self.classifier is None:
            raise ValueError('A very specific bad thing happened')

        return list(self.classifier.predict_proba(x))
    
class ScikitLearnTFDNNRegressor:
    def __init__(self,name,hidden_units,optimizer,steps):
        self.regressor = None
        self.model_dir = None
        self.hidden_units = hidden_units
        self.num_classes = num_classes
        self.optimizer = optimizer
        self.steps = steps
        self.name = name
        
    def __str__(self):
        return "ScikitLearnTFDNNRegressor(name='{}',hidden_units={},optimizer={},steps={})".format(
        self.name,self.hidden_units,self.optimizer,self.steps)

    def fit(self,x,y):
        if self.regressor is None:
            self.model_dir = get_model_dir(self.name,True)

            # Create a deep neural network with 3 hidden layers of 10, 20, 5
            feature_columns = [tf.contrib.layers.real_valued_column("", dimension=x.shape[1])]
            self.regressor = learn.DNNRegressor(
                model_dir= self.model_dir,
                config=tf.contrib.learn.RunConfig(save_checkpoints_secs=30),
                hidden_units=self.hidden_units, feature_columns=feature_columns)

        self.regressor.fit(x,y,steps=self.steps)
        return self

    def predict(self,x):
        if self.regressor is None:
            raise ValueError('A very specific bad thing happened')

        return list(self.regressor.predict(x, as_iterable=True))




# Evaluating Feature Importance

Feature importance tells us how important each of the features (from the feature/import vector are to the prediction of a neural network, or other model.  There are many different ways to evaluate feature importance for neural networks.  The following paper presents a very good (and readable) overview of the various means of evaluating the importance of neural network inputs/features.

Olden, J. D., Joy, M. K., & Death, R. G. (2004). [An accurate comparison of methods for quantifying variable importance in artificial neural networks using simulated data](http://depts.washington.edu/oldenlab/wordpress/wp-content/uploads/2013/03/EcologicalModelling_2004.pdf). *Ecological Modelling*, 178(3), 389-397.

In summary, the following methods are available to neural networks:

* Connection Weights Algorithm
* Partial Derivatives
* Input Perturbation
* Sensitivity Analysis
* Forward Stepwise Addition 
* Improved Stepwise Selection 1
* Backward Stepwise Elimination
* Improved Stepwise Selection

For this class we will use the **Input Perturbation** feature ranking algorithm.  This algorithm will work with any regression or classification network.  implementation of the input perturbation algorithm for scikit-learn is given in the next section. This algorithm is implemented in a function below that will work with any scikit-learn model.

This algorithm was introduced by [Breiman](https://en.wikipedia.org/wiki/Leo_Breiman) in his seminal paper on random forests.  Although he presented this algorithm in conjunction with random forests, it is model-independent and appropriate for any supervised learning model.  This algorithm, known as the input perturbation algorithm, works by evaluating a trained model’s accuracy with each of the inputs individually shuffled from a data set.  Shuffling an input causes it to become useless—effectively removing it from the model. More important inputs will produce a less accurate score when they are removed by shuffling them. This process makes sense, because important features will contribute to the accuracy of the model.  The TensorFlow version of this algorithm is taken from the following paper.

Heaton, J., McElwee, S., & Cannady, J. (May 2017). Early stabilizing feature importance for TensorFlow deep neural networks. In *International Joint Conference on Neural Networks (IJCNN 2017)* (accepted for publication). IEEE.

This algorithm will use logloss to evaluate a classification problem and RMSE for regression.

In [5]:
from sklearn import metrics
import scipy as sp
import numpy as np
import math
from sklearn import metrics

def perturbation_rank(model, x, y, names, regression):
    errors = []

    for i in range(x.shape[1]):
        hold = np.array(x[:, i])
        np.random.shuffle(x[:, i])
        
        if regression:
            pred = model.predict(x_test)
            error = metrics.mean_squared_error(y, pred)
        else:
            pred = model.predict_proba(x)
            error = metrics.log_loss(y, pred)
            
        errors.append(error)
        x[:, i] = hold
        
    max_error = np.max(errors)
    importance = [e/max_error for e in errors]

    data = {'name':names,'error':errors,'importance':importance}
    result = pd.DataFrame(data, columns = ['name','error','importance'])
    result.sort_values(by=['importance'], ascending=[0], inplace=True)
    return result

### Classification Input Perturbation Ranking

In [6]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)


Using TensorFlow backend.


Epoch 00494: early stopping


In [7]:
# Rank the features
from IPython.display import display, HTML

names = df.columns.values[0:-1] # x column names
rank = perturbation_rank(model, x_test, y_test, names, False)
display(rank)

32/38 [========================>.....] - ETA: 0s

,name,error,importance
2,petal_l,1.251176,1.000000
3,petal_w,1.108034,0.885594
1,sepal_w,0.176790,0.141299
0,sepal_l,0.160230,0.128063


### Regression Input Perturbation Ranking

In [8]:
# Rank MPG fields

import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

path = "./data/"

# Set the desired TensorFlow output level for this example
tf.logging.set_verbosity(tf.logging.ERROR)

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
df.drop('name',1,inplace=True)
encode_numeric_zscore(df, 'horsepower')
encode_numeric_zscore(df, 'weight')
encode_numeric_zscore(df, 'cylinders')
encode_numeric_zscore(df, 'displacement')
encode_numeric_zscore(df, 'acceleration')
encode_text_dummy(df, 'origin')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x,y,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)
    

Epoch 00136: early stopping


In [9]:
# Rank the features
from IPython.display import display, HTML

names = df.columns.values[0:-1] # x column names
rank = perturbation_rank(model, x_test, y_test, names, True)
display(rank)

,name,error,importance
5,acceleration,15.779439,1.000000
3,horsepower,15.255577,0.966801
2,displacement,13.450043,0.852378
1,cylinders,12.604163,0.798771
0,mpg,12.052859,0.763833
8,origin-2,11.853414,0.751194
6,year,10.955904,0.694315
4,weight,10.846774,0.687399
7,origin-1,10.536667,0.667747


# The Biological Response Data Set

* [Biological Response Dataset at Kaggle](https://www.kaggle.com/c/bioresponse)
* [1st place interview for Boehringer Ingelheim Biological Response](http://blog.kaggle.com/2012/07/05/1st-place-interview-for-boehringer-ingelheim-biological-response/)

In [10]:
import tensorflow.contrib.learn as skflow
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore
from sklearn.model_selection import KFold
from IPython.display import HTML, display

path = "./data/"

filename_train = os.path.join(path,"bio_train.csv")
filename_test = os.path.join(path,"bio_test.csv")
filename_submit = os.path.join(path,"bio_submit.csv")
df_train = pd.read_csv(filename_train,na_values=['NA','?'])
df_test = pd.read_csv(filename_test,na_values=['NA','?'])

activity_classes = encode_text_index(df_train,'Activity')

#display(df_train)

In [11]:
print(df_train.shape)

(3751, 1777)


### Biological Response with Neural Network

In [12]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow.contrib.learn as skflow
import numpy as np
import sklearn

# Set the desired TensorFlow output level for this example
tf.logging.set_verbosity(tf.logging.ERROR)

# Encode feature vector
x, y = to_xy(df_train,'Activity')
x_submit = df_test.as_matrix().astype(np.float32)
num_classes = len(activity_classes)

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(    
    x, y, test_size=0.25, random_state=42) 

print("Fitting/Training...")
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=0,epochs=1000)
print("Fitting done...")

# Give logloss error
pred = model.predict(x_test)
pred2 = np.argmax(pred,axis=1)
pred = pred[:,1]
# Clip so that min is never exactly 0, max never 1
pred = np.clip(pred,a_min=1e-6,a_max=(1-1e-6)) 
print("Validation logloss: {}".format(sklearn.metrics.log_loss(y_test,pred)))

# Evaluate success using accuracy
pred_submit = pred.copy()
y_test = y_test[:,1]
score = metrics.accuracy_score(y_test, pred2)
print("Validation accuracy score: {}".format(score))

# Build real submit file
pred_submit = model.predict(x_submit)
pred_submit = pred_submit[:,1]

# Clip so that min is never exactly 0, max never 1
pred = np.clip(pred,a_min=1e-6,a_max=(1-1e-6)) 
submit_df = pd.DataFrame({'MoleculeId':[x+1 for x in range(len(pred_submit))],'PredictedProbability':pred_submit})
submit_df.to_csv(filename_submit, index=False)


Fitting/Training...
Epoch 00009: early stopping
Fitting done...
Validation logloss: 0.5504896390007364
Validation accuracy score: 0.7761194029850746


# What Features/Columns are Important 

The following uses perturbation ranking to evaluate the neural network.

In [13]:
# Rank the features
from IPython.display import display, HTML

names = df_train.columns.values[0:-1] # x column names
rank = perturbation_rank(model, x_test, y_test, names, False)
display(rank)

 32/938 [>.............................] - ETA: 0s

,name,error,importance
26,D26,0.595832,1.000000
50,D50,0.565651,0.949347
1099,D1099,0.559131,0.938405
1048,D1048,0.556952,0.934747
1058,D1058,0.556426,0.933864
1035,D1035,0.555929,0.933030
1389,D1389,0.555795,0.932805
1025,D1025,0.555293,0.931963
1092,D1092,0.554741,0.931037
1187,D1187,0.554651,0.930886


### Biological Response with Random Forest

In [14]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier
import sklearn


x, y = to_xy(df_train,'Activity')
y = np.argmax(y,axis=1)
x_test = df_test.as_matrix().astype(np.float32)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(x, y)
pred = rf.predict_proba(x_test)
pred = pred[:,1]
pred_insample = rf.predict_proba(x)
pred_insample = pred_insample[:,1]

submit_df = pd.DataFrame({'MoleculeId':[x+1 for x in range(len(pred))],'PredictedProbability':pred})
submit_df.to_csv(filename_submit, index=False)
print("Insample logloss: {}".format(sklearn.metrics.log_loss(y,pred_insample)))
#display(submit_df)

Insample logloss: 0.1261067880812591


## Random Forest Ranking

In [15]:
# Rank the features
from IPython.display import display, HTML

names = df_train.columns.values[1:] # x column names
rank = perturbation_rank(rf, x, y, names, False)
display(rank)

,name,error,importance
26,D27,0.185490,1.000000
87,D88,0.152633,0.822862
145,D146,0.145366,0.783690
746,D747,0.144338,0.778146
658,D659,0.142734,0.769497
65,D66,0.142582,0.768678
90,D91,0.141818,0.764559
77,D78,0.141775,0.764327
17,D18,0.139714,0.753219
9,D10,0.139666,0.752960


# Neural Network Bagging

Neural networks will typically achieve better results when they are bagged.  Bagging a neural network is a process where the same neural network is trained over and over and the results are averaged together.

In [16]:
import numpy as np
import os
import pandas as pd
import math
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

PATH = "./data/"
SHUFFLE = False
FOLDS = 10

def build_ann(input_size,classes,neurons):
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_size, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def mlogloss(y_test, preds):
    epsilon = 1e-15
    sum = 0
    for row in zip(preds,y_test):
        x = row[0][row[1]]
        x = max(epsilon,x)
        x = min(1-epsilon,x)
        sum+=math.log(x)
    return( (-1/len(preds))*sum)

def stretch(y):
    return (y - y.min()) / (y.max() - y.min())


def blend_ensemble(x, y, x_submit):

    folds = list(StratifiedKFold(y, FOLDS))
    feature_columns = [tf.contrib.layers.real_valued_column("", dimension=x.shape[0])]

    models = [
        KerasClassifier(build_fn=build_ann,neurons=10,input_size=x.shape[1],classes=2),
        KerasClassifier(build_fn=build_ann,neurons=20,input_size=x.shape[1],classes=2),
        KerasClassifier(build_fn=build_ann,neurons=30,input_size=x.shape[1],classes=2)
        ]

    dataset_blend_train = np.zeros((x.shape[0], len(models)))
    dataset_blend_test = np.zeros((x_submit.shape[0], len(models)))

    for j, model in enumerate(models):
        print("Model: {} : {}".format(j, model) )
        fold_sums = np.zeros((x_submit.shape[0], len(folds)))
        total_loss = 0
        for i, (train, test) in enumerate(folds):
            x_train = x[train]
            y_train = y[train]
            x_test = x[test]
            y_test = y[test]
            model.fit(x_train, y_train)
            pred = np.array(model.predict_proba(x_test))
            # pred = model.predict_proba(x_test)
            dataset_blend_train[test, j] = pred[:, 1]
            pred2 = np.array(model.predict_proba(x_submit))
            #fold_sums[:, i] = model.predict_proba(x_submit)[:, 1]
            fold_sums[:, i] = pred2[:, 1]
            loss = mlogloss(y_test, pred)
            total_loss+=loss
            print("Fold #{}: loss={}".format(i,loss))
        print("{}: Mean loss={}".format(model.__class__.__name__,total_loss/len(folds)))
        dataset_blend_test[:, j] = fold_sums.mean(1)

    print()
    print("Blending models.")
    blend = LogisticRegression()
    blend.fit(dataset_blend_train, y)
    return blend.predict_proba(dataset_blend_test)

if __name__ == '__main__':

    np.random.seed(42)  # seed to shuffle the train set

    print("Loading data...")
    filename_train = os.path.join(PATH, "bio_train.csv")
    df_train = pd.read_csv(filename_train, na_values=['NA', '?'])

    filename_submit = os.path.join(PATH, "bio_test.csv")
    df_submit = pd.read_csv(filename_submit, na_values=['NA', '?'])

    predictors = list(df_train.columns.values)
    predictors.remove('Activity')
    x = df_train.as_matrix(predictors)
    y = df_train['Activity']
    x_submit = df_submit.as_matrix()

    if SHUFFLE:
        idx = np.random.permutation(y.size)
        x = x[idx]
        y = y[idx]

    submit_data = blend_ensemble(x, y, x_submit)
    submit_data = stretch(submit_data)

    ####################
    # Build submit file
    ####################
    ids = [id+1 for id in range(submit_data.shape[0])]
    submit_filename = os.path.join(PATH, "bio_submit.csv")
    submit_df = pd.DataFrame({'MoleculeId': ids, 'PredictedProbability': submit_data[:, 1]},
                             columns=['MoleculeId','PredictedProbability'])
    submit_df.to_csv(submit_filename, index=False)

C:\Users\jeffh\Anaconda3\envs\wustl\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Loading data...
Model: 0 : <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000226B5262A90>
Epoch 1/10
3375/3375 [==============================] - 0s - loss: 0.6482     
Epoch 2/10
3375/3375 [==============================] - 0s - loss: 0.5387     
Epoch 3/10
3375/3375 [==============================] - 0s - loss: 0.4807     
Epoch 4/10
3375/3375 [==============================] - 0s - loss: 0.4485     
Epoch 5/10
3375/3375 [==============================] - 0s - loss: 0.4296     
Epoch 6/10
3375/3375 [==============================] - 0s - loss: 0.4107     
Epoch 7/10
3375/3375 [==============================] - 0s - loss: 0.4058     
Epoch 8/10
3375/3375 [==============================] - 0s - loss: 0.3898     
Epoch 9/10
3375/3375 [==============================] - 0s - loss: 0.3857     
Epoch 10/10
2368/2501 [===========================>..] - ETA: 0sFold #0: loss=0.5387259995504785
Epoch 1/10
3375/3375 [==============================] - 0s - loss: 0.6589     
Epoch 2/10


3377/3377 [==============================] - 0s - loss: 0.4843     
Epoch 4/10
3377/3377 [==============================] - 0s - loss: 0.4482     
Epoch 5/10
3377/3377 [==============================] - 0s - loss: 0.4229     
Epoch 6/10
3377/3377 [==============================] - 0s - loss: 0.4099     
Epoch 7/10
3377/3377 [==============================] - 0s - loss: 0.3918     
Epoch 8/10
3377/3377 [==============================] - 0s - loss: 0.3794     
Epoch 9/10
3377/3377 [==============================] - 0s - loss: 0.3623     
Epoch 10/10
2496/2501 [============================>.] - ETA: 0sFold #9: loss=0.566150889706028
KerasClassifier: Mean loss=0.5339929248257194
Model: 1 : <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000226B544C0B8>
Epoch 1/10
3375/3375 [==============================] - 1s - loss: 0.6323     
Epoch 2/10
3375/3375 [==============================] - 0s - loss: 0.5246     
Epoch 3/10
3375/3375 [==============================] - 0s - loss: 0.463

3377/3377 [==============================] - 0s - loss: 0.4195     
Epoch 6/10
3377/3377 [==============================] - 0s - loss: 0.4001     
Epoch 7/10
3377/3377 [==============================] - 0s - loss: 0.3876     
Epoch 8/10
3377/3377 [==============================] - 0s - loss: 0.3754     
Epoch 9/10
3377/3377 [==============================] - 0s - loss: 0.3614     
Epoch 10/10
2272/2501 [==========================>...] - ETA: 0sFold #8: loss=0.49406496336106037
Epoch 1/10
3377/3377 [==============================] - 1s - loss: 0.6308     
Epoch 2/10
3377/3377 [==============================] - 0s - loss: 0.5154     
Epoch 3/10
3377/3377 [==============================] - 0s - loss: 0.4684     
Epoch 4/10
3377/3377 [==============================] - 0s - loss: 0.4357     
Epoch 5/10
3377/3377 [==============================] - 0s - loss: 0.4156     
Epoch 6/10
3377/3377 [==============================] - 0s - loss: 0.4021     
Epoch 7/10
3377/3377 [======================

3377/3377 [==============================] - 0s - loss: 0.4085     
Epoch 7/10
3377/3377 [==============================] - 0s - loss: 0.3888     
Epoch 8/10
3377/3377 [==============================] - 0s - loss: 0.3794     
Epoch 9/10
3377/3377 [==============================] - 0s - loss: 0.3705     
Epoch 10/10
2432/2501 [============================>.] - ETA: 0sFold #7: loss=0.5410842681424806
Epoch 1/10
3377/3377 [==============================] - 1s - loss: 0.6257     
Epoch 2/10
3377/3377 [==============================] - 0s - loss: 0.5125     
Epoch 3/10
3377/3377 [==============================] - 0s - loss: 0.4632     
Epoch 4/10
3377/3377 [==============================] - 1s - loss: 0.4314     
Epoch 5/10
3377/3377 [==============================] - 0s - loss: 0.4116     
Epoch 6/10
3377/3377 [==============================] - 0s - loss: 0.3966     
Epoch 7/10
3377/3377 [==============================] - 0s - loss: 0.3854     
Epoch 8/10
3377/3377 [=======================

# Neural Network Ensemble

A neural network ensemble combines neural network predictions with other models.  The exact blend of all of these models is determined by logistic regression.  The following code performs this blend for a classification.

In [17]:
import numpy as np
import os
import pandas as pd
import math
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

PATH = "./data/"
SHUFFLE = False
FOLDS = 10

def build_ann(input_size,classes,neurons):
    model = Sequential()
    model.add(Dense(neurons, input_dim=input_size, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.add(Dense(classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def mlogloss(y_test, preds):
    epsilon = 1e-15
    sum = 0
    for row in zip(preds,y_test):
        x = row[0][row[1]]
        x = max(epsilon,x)
        x = min(1-epsilon,x)
        sum+=math.log(x)
    return( (-1/len(preds))*sum)

def stretch(y):
    return (y - y.min()) / (y.max() - y.min())


def blend_ensemble(x, y, x_submit):

    folds = list(StratifiedKFold(y, FOLDS))
    feature_columns = [tf.contrib.layers.real_valued_column("", dimension=x.shape[0])]

    models = [
        KerasClassifier(build_fn=build_ann,neurons=20,input_size=x.shape[1],classes=2),
        KNeighborsClassifier(n_neighbors=3),
        RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        RandomForestClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini'),
        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy'),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=50)]

    dataset_blend_train = np.zeros((x.shape[0], len(models)))
    dataset_blend_test = np.zeros((x_submit.shape[0], len(models)))

    for j, model in enumerate(models):
        print("Model: {} : {}".format(j, model) )
        fold_sums = np.zeros((x_submit.shape[0], len(folds)))
        total_loss = 0
        for i, (train, test) in enumerate(folds):
            x_train = x[train]
            y_train = y[train]
            x_test = x[test]
            y_test = y[test]
            model.fit(x_train, y_train)
            pred = np.array(model.predict_proba(x_test))
            # pred = model.predict_proba(x_test)
            dataset_blend_train[test, j] = pred[:, 1]
            pred2 = np.array(model.predict_proba(x_submit))
            #fold_sums[:, i] = model.predict_proba(x_submit)[:, 1]
            fold_sums[:, i] = pred2[:, 1]
            loss = mlogloss(y_test, pred)
            total_loss+=loss
            print("Fold #{}: loss={}".format(i,loss))
        print("{}: Mean loss={}".format(model.__class__.__name__,total_loss/len(folds)))
        dataset_blend_test[:, j] = fold_sums.mean(1)

    print()
    print("Blending models.")
    blend = LogisticRegression()
    blend.fit(dataset_blend_train, y)
    return blend.predict_proba(dataset_blend_test)

if __name__ == '__main__':

    np.random.seed(42)  # seed to shuffle the train set

    print("Loading data...")
    filename_train = os.path.join(PATH, "bio_train.csv")
    df_train = pd.read_csv(filename_train, na_values=['NA', '?'])

    filename_submit = os.path.join(PATH, "bio_test.csv")
    df_submit = pd.read_csv(filename_submit, na_values=['NA', '?'])

    predictors = list(df_train.columns.values)
    predictors.remove('Activity')
    x = df_train.as_matrix(predictors)
    y = df_train['Activity']
    x_submit = df_submit.as_matrix()

    if SHUFFLE:
        idx = np.random.permutation(y.size)
        x = x[idx]
        y = y[idx]

    submit_data = blend_ensemble(x, y, x_submit)
    submit_data = stretch(submit_data)

    ####################
    # Build submit file
    ####################
    ids = [id+1 for id in range(submit_data.shape[0])]
    submit_filename = os.path.join(PATH, "bio_submit.csv")
    submit_df = pd.DataFrame({'MoleculeId': ids, 'PredictedProbability': submit_data[:, 1]},
                             columns=['MoleculeId','PredictedProbability'])
    submit_df.to_csv(submit_filename, index=False)

Loading data...
Model: 0 : <keras.wrappers.scikit_learn.KerasClassifier object at 0x00000226B3FF6898>
Epoch 1/10
3375/3375 [==============================] - 1s - loss: 0.6450     
Epoch 2/10
3375/3375 [==============================] - 0s - loss: 0.5298     
Epoch 3/10
3375/3375 [==============================] - 0s - loss: 0.4709     
Epoch 4/10
3375/3375 [==============================] - 0s - loss: 0.4391     
Epoch 5/10
3375/3375 [==============================] - 0s - loss: 0.4192     
Epoch 6/10
3375/3375 [==============================] - 0s - loss: 0.3982     
Epoch 7/10
3375/3375 [==============================] - 0s - loss: 0.3914     
Epoch 8/10
3375/3375 [==============================] - 0s - loss: 0.3731     
Epoch 9/10
3375/3375 [==============================] - 0s - loss: 0.3688     
Epoch 10/10
2400/2501 [===========================>..] - ETA: 0sFold #0: loss=0.5338064958666865
Epoch 1/10
3375/3375 [==============================] - 1s - loss: 0.6497     
Epoch 2/10


3377/3377 [==============================] - 0s - loss: 0.4660     
Epoch 4/10
3377/3377 [==============================] - 0s - loss: 0.4337     
Epoch 5/10
3377/3377 [==============================] - 0s - loss: 0.4062     
Epoch 6/10
3377/3377 [==============================] - 0s - loss: 0.3880     
Epoch 7/10
3377/3377 [==============================] - 0s - loss: 0.3673     
Epoch 8/10
3377/3377 [==============================] - 0s - loss: 0.3568     
Epoch 9/10
3377/3377 [==============================] - 0s - loss: 0.3334     
Epoch 10/10
2400/2501 [===========================>..] - ETA: 0sFold #9: loss=0.5894619616228683
KerasClassifier: Mean loss=0.5410438925071015
Model: 1 : KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')
Fold #0: loss=3.606678388314123
Fold #1: loss=2.2197228940978317
Fold #2: loss=3.6717523663107237
Fold #3: loss=2.5045156203944594
Fold #4: 